# AIIR Project - AI Mario
This jupyter notebook contains the application of nueral network and reinforcement learning algorithms learnt from the tutorials to simulate Mario completing a variety of levels in a pybullet gym environment.


## Mario Environment
We use a Super Mario Bros environment (https://pypi.org/project/gym-super-mario-bros/) with a continuous state space and discrete action space. The goal of this activity is to complete Mario levels as fast as possible. Episodes end when Mario reaches the end of the level, if Mario dies, or if a certain time as elapsed.

### Action Space
- 0: No Movement
- 1: Move Right
- 2: Move Right + Jump
- 3: Move Right + Speed Up
- 4: Move Right + Jump + Speed Up
- 5: Jump
- 6: Move Left
- 7: Move Left + Jump
- 8: Move Left + Speed Up
- 9: Move Left + Jump + Speed Up
- 10: Down
- 11: Up

### Observation Space
The info dictionary returned by step contains the following:
| Key | Unit | Description |
| --- | ---- | ----------- |
| coins | int | Number of collected coins |
| flag_get | bool | True if Mario reached a flag |
| life | int | Number of lives left |
| score | int | Cumulative in-game score |
| stage | int | Current stage |
| status | str | Mario's status/power |
| time | int | Time left on the clock |
| world | int | Current world |
| x_pos | int | Mario's x position in the stage |
| y_pos | int | Mario's y position in the stage |

### Rewards
| Feature | Description | Value when Positive | Value when Negative | Value when Equal |
|---------|-------------|---------------------|---------------------|------------------|
| Difference in agent x values between states | Controls agent's movement | Moving right | Moving left | Not moving |
| Time difference in the game clock between frames | Prevents agent from staying still | - | Clock ticks | Clock doesn't tick |
| Death Penalty | Discourages agent from death | - | Agent dead | Agent alive |
| - | - | - | - | - |

## Installation Guide
For installing the Super Mario Bros gym environment package, use the following command using a python 3.8 kernel:

In [ ]:
%pip install gym-super-mario-bros

Importing the necessary packages and following helper function to display video runs within jupyter notebook

In [ ]:
import os
os.environ['PYVIRTUALDISPLAY_DISPLAYFD'] = '0' 

import gym
import pybullet as p
import matplotlib.pyplot as plt
from pyvirtualdisplay import Display
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import torch
import random

display = Display(visible=0, size=(400, 300))
display.start()

def display_video(frames, framerate=30):
  """Generates video from `frames`.

  Args:
    frames (ndarray): Array of shape (n_frames, height, width, 3).
    framerate (int): Frame rate in units of Hz.

  Returns:
    Display object.
  """
  height, width, _ = frames[0].shape
  dpi = 70
  orig_backend = matplotlib.get_backend()
  matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
  fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
  matplotlib.use(orig_backend)  # Switch back to the original backend.
  ax.set_axis_off()
  ax.set_aspect('equal')
  ax.set_position([0, 0, 1, 1])
  im = ax.imshow(frames[0])
  def update(frame):
    im.set_data(frame)
    return [im]
  interval = 1000/framerate
  anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                  interval=interval, blit=True, repeat=False)
  return HTML(anim.to_html5_video())

## Hyperparameters

In [ ]:
# Hyperparameters
EPISODES = 2500                  # Number of episodes to run for training
LEARNING_RATE = 0.00025          # Learning rate for optimizing neural network weights
MEM_SIZE = 50000                 # Maximum size of replay memory
REPLAY_START_SIZE = 10000        # Amount of samples to fill replay memory before training
BATCH_SIZE = 32                  # Number of samples to draw from replay memory for training
GAMMA = 0.99                     # Discount factor for future rewards
EPSILON_START = 0.1              # Starting exploration rate
EPSILON_END = 0.0001             # Final exploration rate
EPSILON_DECAY = 4 * MEM_SIZE     # Decay rate for exploration rate
MEM_RETAIN = 0.1                 # Percentage of memory to retain on each episode
NETWORK_UPDATE_ITERS = 5000      # Number of steps to update target network

FC1_DIMS = 128                   # Number of neurons in our MLP's first hidden layer
FC2_DIMS = 128                   # Number of neurons in our MLP's second hidden layer

# Metrics for displaying training status
best_reward = 0
average_reward = 0
episode_history = []
episode_reward_history = []
np.bool = np.bool_

## Neural Network
Below is the class definition for a neural network used to approximate Q-values for the use within a reinforcement learning framework.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Neural network class used to approximate Q-values within the Mario environment
class Network(torch.nn.Module):
    def __init__(self, env):
        super(Network, self).__init__()  # Inheriting from torch.nn.Module

        self.input_shape = env.observation_space.shape  # Getting shape of observation space
        self.action_space = env.action_space.n          # Getting number of actions in action space

        # Defining the layers of the neural network
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(*self.input_shape, FC1_DIMS),   # 
            torch.nn.ReLU(),                                # 
            torch.nn.Linear(FC1_DIMS, FC2_DIMS),            #
            torch.nn.ReLU(),                                #
            torch.nn.Linear(FC2_DIMS, self.action_space)    #
        )

        self.optimizer = optim.Adam(self.parameters(), lr=LEARNING_RATE)  # Optimizer
        self.loss = nn.MSELoss()  # Loss Function

    def forward(self, x):
        return self.layers(x)  # Forward pass through the network

## Reinforcement Learning Framework

## Training the AI
The following code uses the above Neural Network and Reinforcement Learning framework to train the AI to play Super Mario Bros on a variety of its levels.

In [ ]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT
import gym

env = gym_super_mario_bros.make('SuperMarioBrosRandomStages-v0', apply_api_compatibility=True, render_mode="rgb_array")
env = JoypadSpace(env, COMPLEX_MOVEMENT)

state = env.reset()
frames = []
frames.append(env.render())

for step in range(EPISODES/EPISODES * 200):
    action = env.action_space.sample()
    state_, reward, done, truncated, info = env.step(action)
    frames.append(np.copy(env.render()))

    if done:
        break

env.close()
display_video(frames)

## Testing the AI
The following code tests the AI policy generated during training in random Super Mario Bros levels.

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBrosRandomStages-v0', apply_api_compatibility=True, render_mode="rgb_array")
env = JoypadSpace(env, COMPLEX_MOVEMENT)

